<a href="https://colab.research.google.com/github/mikvikpik/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
# importing all databases
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import chisquare
from scipy import stats

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',
                 header=None, names=['A1','A2','A3','A4','A5','A6','A7','A8','A9',
                                     'A10','A11','A12','A13','A14','A15','A16'])
df.head()

# A1, A16 are seemingly binomial and can be encoded to 1/0 

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [3]:
# Checking shape to validate appropriate number of observations
df.shape

(690, 16)

## Part 2 - Exploring data, Testing hypotheses

**The only thing we really know about this data is that A16 is the class label.**

Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. 

**In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`). A16**

**For the 6 continuous features(A2,3,8,11,14,15), how are they different when split between the two class labels?** 

Choose two features to run t-tests (again split by class label)A16 - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

**For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them.**

[pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, **try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.**

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

**Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.**

This is challenging, so manage your time and aim for a **baseline of at least running two t-tests and two Chi-squared tests before polishing.** And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [4]:
# change a/b values in A1 to 1/0
df['A1'] = df['A1'].replace(to_replace='a', value=0.0)
df = df.replace(to_replace='b', value=1.0)
df = df.replace(to_replace='?', value=float('nan'))
df.head()


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1.0,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,0.0,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,0.0,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,1.0,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,1.0,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [5]:
#Checking A1 for null values
df['A1'].isnull().sum()

12

In [6]:
# Checking values in A16
df['A16'].value_counts()

-    383
+    307
Name: A16, dtype: int64

In [7]:
# changing +/- values to 1.0 for class value A16
df = df.replace(to_replace='-', value=0.0)
df = df.replace(to_replace='+', value=1.0)

df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1.0,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,1.0
1,0.0,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,1.0
2,0.0,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,1.0
3,1.0,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,1.0
4,1.0,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,1.0


In [8]:
# Checking for null values in continuous attributes
df['A2'].isnull().sum(),df['A3'].isnull().sum(),df['A8'].isnull().sum(),df['A11'].isnull().sum(),df['A14'].isnull().sum(),df['A15'].isnull().sum(),

(12, 0, 0, 0, 13, 0)

In [0]:
# Converting A2 from strings to floats
df['A2'] = df['A2'].astype(float)


In [18]:
# Creating 2 dfs of df_class_1, df_class_0 to start comparison of data
df_class_1 = df[df['A16']==1.0]
df_class_0 = df[df['A16']==0.0]

print(df_class_1.head())
print(df_class_0.head())

    A1     A2     A3 A4 A5 A6 A7    A8 A9 A10  A11 A12 A13    A14  A15  A16
0  1.0  30.83  0.000  u  g  w  v  1.25  t   t    1   f   g  00202    0  1.0
1  0.0  58.67  4.460  u  g  q  h  3.04  t   t    6   f   g  00043  560  1.0
2  0.0  24.50  0.500  u  g  q  h  1.50  t   f    0   f   g  00280  824  1.0
3  1.0  27.83  1.540  u  g  w  v  3.75  t   t    5   t   g  00100    3  1.0
4  1.0  20.17  5.625  u  g  w  v  1.71  t   f    0   f   s  00120    0  1.0
     A1     A2   A3 A4 A5  A6  A7      A8 A9 A10  A11 A12 A13    A14  A15  A16
70  1.0  32.33  7.5  u  g   e  bb   1.585  t   f    0   t   s  00420    0  0.0
71  1.0  34.83  4.0  u  g   d  bb  12.500  t   f    0   t   g    NaN    0  0.0
72  0.0  38.58  5.0  u  g  cc   v  13.500  t   f    0   t   g  00980    0  0.0
73  1.0  44.25  0.5  u  g   m   v  10.750  t   f    0   f   s  00400    0  0.0
74  1.0  44.83  7.0  y  p   c   v   1.625  f   f    0   f   g  00160    2  0.0


In [20]:
""" checking separate dataframes for obvious statistical variances
    Mostly means and std are lightly varied, however A11 and A15 have differences
    Notably, 11 and 15 have at nearly half or more than half of data with 0 data
    A3 has relatively more closely aligned data and 
df_class_0.describe()

,A1,A2,A3,A8,A11,A15,A16
count,374.000000,373.000000,383.000000,383.000000,383.000000,383.000000,383.0
mean,0.700535,29.808231,3.839948,1.257924,0.631854,198.605744,0.0
std,0.458637,10.919291,4.337662,2.120481,1.900049,671.608839,0.0
min,0.000000,15.170000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,22.000000,0.835000,0.125000,0.000000,0.000000,0.0
50%,1.000000,27.330000,2.210000,0.415000,0.000000,1.000000,0.0
75%,1.000000,34.830000,5.000000,1.500000,0.000000,67.000000,0.0
max,1.000000,80.250000,26.335000,13.875000,20.000000,5552.000000,0.0


In [21]:
df_class_1.describe()

,A1,A2,A3,A8,A11,A15,A16
count,304.000000,305.000000,307.000000,307.000000,307.000000,307.000000,307.0
mean,0.677632,33.720492,5.904951,3.427899,4.605863,2038.859935,1.0
std,0.468154,12.809641,5.471485,4.120792,6.320242,7659.763941,0.0
min,0.000000,13.750000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,23.170000,1.500000,0.750000,0.000000,0.000000,1.0
50%,1.000000,30.500000,4.460000,2.000000,3.000000,221.000000,1.0
75%,1.000000,41.330000,9.520000,5.000000,7.000000,1209.000000,1.0
max,1.000000,76.750000,28.000000,28.500000,67.000000,100000.000000,1.0


##T-Tests

In [22]:
ttest_ind(df_class_0['A11'],df_class_1['A11'], nan_policy='omit')
""" 
null hypothesis would be that A11 attribute has no bearing on class label
alternative hypothesis would be that A11 attribute is a fundamental 
difference of class label A16 
this is based on the known fact of widely differing means and std
""""""

Ttest_indResult(statistic=-11.667004222431277, pvalue=7.957718568079967e-29)

In [23]:
ttest_ind(df_class_0['A15'],df_class_1['A15'], nan_policy='omit')
""" 
null hypothesis would be that A15 attribute has no bearing on class label
alternative hypothesis would be that A15 attribute is a fundamental 
difference of class label A16 
""""""

Ttest_indResult(statistic=-4.680216020964486, pvalue=3.4520256956287944e-06)

In [26]:
ttest_ind(df_class_0['A2'],df_class_1['A2'], nan_policy='omit')
""" 
null hypothesis would be that A2 attribute has no bearing on class label
alternative hypothesis would be that A2 attribute is a fundamental 
difference of class label A16 
""""""

Ttest_indResult(statistic=-4.2922156166315535, pvalue=2.0276637071781407e-05)

In [27]:
ttest_ind(df_class_0['A3'],df_class_1['A3'], nan_policy='omit')
""" 
null hypothesis would be that A3 attribute has no bearing on class label
alternative hypothesis would be that A3 attribute is a fundamental 
difference of class label A16 
""""""

Ttest_indResult(statistic=-5.52998337614816, pvalue=4.551680702308068e-08)

In [28]:
ttest_ind(df_class_0['A8'],df_class_1['A8'], nan_policy='omit')
""" 
null hypothesis would be that A8 attribute has no bearing on class label
alternative hypothesis would be that A8 attribute is a fundamental 
difference of class label A16 
""""""

Ttest_indResult(statistic=-8.935819983773698, pvalue=3.6710537401601785e-18)

##T-Test Summary
All T-test p-values are less than the .05 range required and are therefore all null hypotheses are **REJECTED** because there are significant differences between the means of the df_class_0 and df_class_1 and the attributes tested above.

##Chi-squared Tests

In [30]:
pd.crosstab(df['A16'], df['A4'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A4,l,u,y,All
A16,,,,
0.0,0,263,118,381
1.0,2,256,45,303
All,2,519,163,684


In [47]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A4']))

(26.234074966202144,
 2.010680204180363e-06,
 2,
 array([[  1.11403509, 289.09210526,  90.79385965],
        [  0.88596491, 229.90789474,  72.20614035]]))

In [31]:
pd.crosstab(df['A16'], df['A5'], margins=True)
"""
null hypothesis is there is no bearing on A5 value and determination of credit 
alternative hypothesis is there is a weighted difference of A5 value and credit
"""

A5,g,gg,p,All
A16,,,,
0.0,263,0,118,381
1.0,256,2,45,303
All,519,2,163,684


In [48]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A5']))

(26.234074966202144,
 2.010680204180363e-06,
 2,
 array([[289.09210526,   1.11403509,  90.79385965],
        [229.90789474,   0.88596491,  72.20614035]]))

In [32]:
pd.crosstab(df['A16'], df['A6'], margins=True)
"""
null hypothesis is there is no bearing on A5 value and determination of credit 
alternative hypothesis is there is a weighted difference of A5 value and credit
"""

A6,aa,c,cc,d,e,ff,i,j,k,m,q,r,w,x,All
A16,,,,,,,,,,,,,,,
0.0,35,75,12,23,11,46,45,7,37,22,27,1,31,6,378
1.0,19,62,29,7,14,7,14,3,14,16,51,2,33,32,303
All,54,137,41,30,25,53,59,10,51,38,78,3,64,38,681


In [49]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A6']))

(98.32520342679135,
 3.4999300402715717e-15,
 13,
 array([[29.97356828, 76.04405286, 22.75770925, 16.65198238, 13.87665198,
         29.4185022 , 32.74889868,  5.55066079, 28.30837004, 21.09251101,
         43.29515419,  1.66519824, 35.52422907, 21.09251101],
        [24.02643172, 60.95594714, 18.24229075, 13.34801762, 11.12334802,
         23.5814978 , 26.25110132,  4.44933921, 22.69162996, 16.90748899,
         34.70484581,  1.33480176, 28.47577093, 16.90748899]]))

In [33]:
pd.crosstab(df['A16'], df['A7'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A7,bb,dd,ff,h,j,n,o,v,z,All
A16,,,,,,,,,,
0.0,34,4,49,51,5,2,1,230,2,378
1.0,25,2,8,87,3,2,1,169,6,303
All,59,6,57,138,8,4,2,399,8,681


In [50]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A7']))

(45.03420714024057,
 3.6254528723722447e-07,
 8,
 array([[ 32.74889868,   3.33039648,  31.63876652,  76.59911894,
           4.44052863,   2.22026432,   1.11013216, 221.47136564,
           4.44052863],
        [ 26.25110132,   2.66960352,  25.36123348,  61.40088106,
           3.55947137,   1.77973568,   0.88986784, 177.52863436,
           3.55947137]]))

In [34]:
pd.crosstab(df['A16'], df['A9'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A9,f,t,All
A16,,,
0.0,306,77,383
1.0,23,284,307
All,329,361,690


In [51]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A9']))

(355.2038167412799,
 3.1185900878457007e-79,
 1,
 array([[182.61884058, 200.38115942],
        [146.38115942, 160.61884058]]))

In [35]:
pd.crosstab(df['A16'], df['A10'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A10,f,t,All
A16,,,
0.0,297,86,383
1.0,98,209,307
All,395,295,690


In [52]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A10']))

(143.06956205083148,
 5.6757273745274924e-33,
 1,
 array([[219.25362319, 163.74637681],
        [175.74637681, 131.25362319]]))

In [36]:
pd.crosstab(df['A16'], df['A12'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A12,f,t,All
A16,,,
0.0,213,170,383
1.0,161,146,307
All,374,316,690


In [53]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A12']))

(0.568273300792113,
 0.45094587758631943,
 1,
 array([[207.59710145, 175.40289855],
        [166.40289855, 140.59710145]]))

In [37]:
pd.crosstab(df['A16'], df['A13'], margins=True)
"""
null hypothesis is there is no bearing on A4 value and determination of credit 
alternative hypothesis is there is a weighted difference of A4 value and credit
"""

A13,g,p,s,All
A16,,,,
0.0,338,3,42,383
1.0,287,5,15,307
All,625,8,57,690


In [54]:
stats.chi2_contingency(pd.crosstab(df['A16'], df['A13']))

(9.191570451545381,
 0.010094291370456372,
 2,
 array([[346.92028986,   4.44057971,  31.63913043],
        [278.07971014,   3.55942029,  25.36086957]]))

## Chi-squared Summary

Categories 4, 5, 6, 7, 9, 10 p-values are less than the .05 range required and are therefore all null hypotheses of these categories are **REJECTED** because there are significant differences between the means of the 0.0 and 1.0 values of the A16 class labels.

Category 13 is worth noting because of the .01 p-value that is higher that the near 0 p-values of the other categories do have. This null hypothesis is **REJECTED** because of it's significant value to show determination of separation of the means of the A16 class labels.

Category 12 has a high p-value of .451 and therefore has a null hypothesis that is a **FAILURE TO REJECT** because of the widened area of inclusion.




## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?

**Answer:** Overall, the continuous variables that held a higher mean were included in the A16(1.0) category and looked to require higher scores to be included. This made a significant statistic noting that all tests skewed toward the higher A16(1.0) and could not be made indeterminately. 

In conclusion, the relatively higher scores across the continuous attributes, the more likely to be included in the A16(1.0) group. As with the opposite, the relatively lower scores were included in the A16(0.0) group.


- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?

**Answer:** Overall, the categorical variables were all skewed to have determinate values except for A12 which numerically speaking seemed more closely aligned to a representative sample than skewed towards one A16 group or another. Many of the values showed that a particular value in an attribute made a significant difference in being part of the A16(1.0) group over the A16(0.0) group and vice-versa. 

In conclusion, many of the findings in A16(1.0) can be given to attributes that have a higher propensity towards to be valued while many findings of A16(0.0) have the attributes of the opposite.


- What was the most challenging part of this sprint challenge?

**Answer:** I didn't find a need for visualization and am concerned if there was a need for it that I missed or miscomprehended.  Being able to comprehend the data without further scope of what the data meant was a challenge at first, but taking statistical data takes much of the guesswork out of it by comparing it to significant meaning.



Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

*Your words here!*